In [110]:
import pandas as pd
fpath = "/Users/liuhaohe/Downloads/test/20-2021-12-01-gt-and_2021_12_04_14_41_27_301 (1).xlsx"
sh_name = pd.read_excel(fpath, sheet_name = "Analysis Summary-GeneralSe")
names = [name.strip() for name in sh_name.at[4, "Unnamed: 3"].split("vs.")]
baseline_name = names[0]
another_name = names[1]
FILTER_OUT_SPAM_3 = True
FILTER_OUT_SPAM_2_THREE_TIMES = True
FILTER_ZERO_RATIO_THRESHOLD=0.01
print("Process", fpath)
sh_spam = pd.read_excel(fpath, sheet_name = "SpamCases")
sh_details = pd.read_excel(fpath, sheet_name = "Details")
score_no_filter_no_spam = sh_details.groupby("Domain").get_group("GeneralSentence").groupby("Voice name").get_group(another_name)["Score"].mean()
score_no_filter_with_spam = sh_details.groupby("Voice name").get_group(another_name)["Score"].mean()
print("Mean CMOS score without filter is %f. Calculated with spam: " % score_no_filter_with_spam)
print("Mean CMOS score without filter is %f. Calculated without spam: " % score_no_filter_no_spam)

Process /Users/liuhaohe/Downloads/test/20-2021-12-03-vits-a_2021_12_10_14_20_33_038 (1).xlsx
Mean CMOS score without filter is -0.009615. Calculated with spam: 
Mean CMOS score without filter is -0.047500. Calculated without spam: 


In [114]:
len(sh_details)

1040

In [107]:
all_judgers = sh_spam["JudgeId"].tolist()
filted_type_1 = sh_spam[sh_spam["Score(+-3)"]>0]["JudgeId"].tolist()
print("Judgers: ",filted_type_1," scoring on spam case is +/-3")
filted_type_2 = sh_spam[sh_spam["Score(+-2)"]>=3]["JudgeId"].tolist()
print("Judgers: ",filted_type_2," scoring on spam case is +/-2 for 3 times or more")
filted_type_3 = sh_spam[sh_spam["Percent(Score:0)"].str.strip("%").astype(float)/100 < FILTER_ZERO_RATIO_THRESHOLD]["JudgeId"].tolist()
print("Judgers: ", filted_type_3, "zero score ratio on spam case is < %.1f percent" % (FILTER_ZERO_RATIO_THRESHOLD * 100))
excluded_judger = []
excluded_judger.extend(filted_type_1+filted_type_2+filted_type_3)
excluded_judger = list(set(excluded_judger))
remain_judgers = [x for x in all_judgers if x not in excluded_judger]
print("--> Finally we only consider these judgers:", remain_judgers)

Judgers:  [976829]  scoring on spam case is +/-3
Judgers:  [974537, 976871, 976909, 1001898, 1005222, 1092762]  scoring on spam case is +/-2 for 3 times or more
Judgers:  [191674, 794641, 918740, 922674, 928708, 930711, 936107, 944105, 950743, 951935, 962761, 974355, 974371, 974477, 974537, 975138, 975183, 975701, 975723, 976195, 976663, 976693, 976829, 976871, 976909, 990528, 995540, 1001898, 1003559, 1005222, 1035315, 1062749, 1082866, 1082886, 1092727, 1092762, 1092780, 1092820, 1092834, 1092865, 1093255, 1093276, 1100227] zero score ratio on spam case is < 1.0 percent
--> Finally we only consider these judgers: [478458, 717963, 933180, 1092711, 1092779]


In [108]:
sh_filtered_details = sh_details[sh_details["LE name"].isin(remain_judgers)]
score_filtered_no_spam = sh_filtered_details.groupby("Domain").get_group("GeneralSentence").groupby("Voice name").get_group(another_name)["Score"].mean()
score_filtered_with_spam = sh_filtered_details.groupby("Voice name").get_group(another_name)["Score"].mean()
print("Mean CMOS score with filter is %f. Calculated with spam." % score_filtered_with_spam)
print("Mean CMOS score with filter is %f. Calculated without spam." % score_filtered_no_spam)

Mean CMOS score with filter is -0.119048. Calculated with spam.
Mean CMOS score with filter is -0.185185. Calculated without spam.


In [109]:
data = pd.DataFrame(
       {"Mean CMOS with spam no filter":[score_no_filter_with_spam], 
        "Mean CMOS with spam with filter":[score_filtered_with_spam], 
        "Mean CMOS no spam no filter":[score_no_filter_no_spam], 
        "Mean CMOS no spam with filter":[score_filtered_no_spam], 
       }
       )
with pd.ExcelWriter(fpath, mode='a', engine='openpyxl', if_sheet_exists="replace") as writer:
       data.to_excel(writer, sheet_name="filted_result")